In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver import ActionChains

import time
import pandas as pd

# URL 분석
- 카테고리 페이지는 `middle_category_id`라는 인자를 통해 결정된다.
    - [상의](https://zigzag.kr/categories/-1?title=%EC%9D%98%EB%A5%98&category_id=-1&middle_category_id=474): `474`
    - [하의 (팬츠)](https://zigzag.kr/categories/-1?title=%EC%9D%98%EB%A5%98&category_id=-1&middle_category_id=547): `547`
- 추후 카테고리가 추가될 때의 편의성을 위해, `dict`에 카테고리 아이디를 담아 넣는다.
    - 필요하다면, 이를 config 파일로 빼내기
- `리뷰많은순`으로 정렬 순서를 바꿀 필요가 있는데, 이는 맨 뒤에 `sort` 인자에 `201`을 넣어주면 해결된다.

In [7]:
zigzag_url = "https://zigzag.kr/categories/-1?title=%EC%9D%98%EB%A5%98&category_id=-1&middle_category_id={id}&sort=201"
id_dict = {
    '상의' : '474',
    '하의' : '547'
}

for category, id in id_dict.items():
    print(category, ":", zigzag_url.format(id=id))

상의 : https://zigzag.kr/categories/-1?title=%EC%9D%98%EB%A5%98&category_id=-1&middle_category_id=474&sort=201
하의 : https://zigzag.kr/categories/-1?title=%EC%9D%98%EB%A5%98&category_id=-1&middle_category_id=547&sort=201


# 크롤링 순서
~~1. 정렬 순서를 `리뷰많은순`으로 변경한다.~~
    - URL로 정렬 방식 변경 가능
~~2. 상품 링크를 원하는 수량만큼 불러온다. (테스트의 경우 시간 문제로 10개만)~~
3. 링크에 들어간다.
4. 상품과 관련된 데이터 크롤링
    - 상품 이름
    - 상품 가격 (할인 제외가격)
    - 카테고리 (`id_dict`의 key)
    - 이미지 url (한장)
    - 상세설명 (우선 스킵)
    - 사이즈 정보 (리스트로)
5. 상품 리뷰와 관련된 데이터 크롤링
    - 리뷰아이디
    - 성별, 키, 몸무게
    - 구매 관련 정보
    - 핏 관련 정보
    - 기타 후기


## 상품 링크 불러오기

In [64]:
def get_product_links(driver, url, max_num=2):
    link_set = set()
    link_list = []
    wait = WebDriverWait(driver, 10)
    driver.get(url)
    xpath = '//*[@id="__next"]/div[1]/div/main/section[2]/div/div/div/div/div[1]/div/div[{i}]/div/a'
    i = 1
    while len(link_set) < max_num:
        try:
            element = wait.until(EC.presence_of_element_located((By.XPATH, xpath.format(i=i))))
            href = element.get_attribute('href')
            if href not in link_set:
                link_list.append(href)
            link_set.add(href)
            i += 1
        except (NoSuchElementException, TimeoutException) as e:
            print(e)
            continue

    return link_list

In [65]:
## get_product_links test
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
for _, id in id_dict.items():
    url = zigzag_url.format(id=id)
    links = get_product_links(driver, url, 3)
    print(links)
    driver.implicitly_wait(1)
    print()
driver.close()

['https://zigzag.kr/catalog/products/112538672', 'https://zigzag.kr/catalog/products/106711935', 'https://zigzag.kr/catalog/products/100511186']

['https://zigzag.kr/catalog/products/140751630', 'https://zigzag.kr/catalog/products/113041967', 'https://zigzag.kr/catalog/products/100341594']



## 링크 진입 후, 원하는 데이터 크롤링
- XPATH 정리
    - 상품 이름
        - `//*[@id="__next"]/div[1]/div/div/div[4]/h1`
    - 상품 가격 (할인 제외가격)
        - `//*[@id="__next"]/div[1]/div/div/div[6]/div[1]/div/div[1]/div[2]`
    - 카테고리 (`id_dict`의 key)
    - 이미지 url (한장)
        - `//*[@id="__next"]/div[1]/div/div/div[1]/div[1]/div/div/div[1]/div[1]/div/div/picture/img`
    - 상세설명 (우선 스킵)
    - 사이즈 정보 (리스트로)
        - 색상별로 사이즈가 다 달라서 고민이 필요
    - 색상 정보

In [71]:
## 우선은 이름, 가격, 이미지url만
def extract_product_info(driver, url):
    try:
        driver.get(url)
        # driver.implicitly_wait(5)
        # name = driver.find_element(By.XPATH, '//*[@id="__next"]/div[1]/div/div/div[4]/h1')
        wait = WebDriverWait(driver, 10)
        name = wait.until(EC.presence_of_element_located((By.CLASS_NAME, 'BODY_15 REGULAR css-1qjogoj e1wgb8lp0')))
        # price = wait.until(EC.presence_of_element_located((\
        #     By.XPATH, '//*[@id="__next"]/div[1]/div/div/div[6]/div[1]/div/div[1]/div[2]')))
        # image_url = wait.until(EC.presence_of_element_located((\
        #     By.XPATH, '//*[@id="__next"]/div[1]/div/div/div[1]/div[1]/div/div/div[1]/div[1]/div/div/picture/img')))

        return {
            'name': name.text,
            # 'price': price.get_attribute('div'),
            # 'image_url': image_url.get_attribute('')
        }
    except (NoSuchElementException, TimeoutError) as e:
        print(e)
    finally:
        driver.close()

In [74]:
## 지그재그 리뷰는 일단 되는듯

with webdriver.Chrome(service=Service(ChromeDriverManager().install())) as driver:
    driver.get("https://zigzag.kr/review/list/111217060")
    actions = driver.find_element(By.CSS_SELECTOR, 'body')
    actions.send_keys(Keys.END)
    driver.implicitly_wait(1)
    actions.send_keys(Keys.END)
    driver.implicitly_wait(1)
    actions.send_keys(Keys.END)
    driver.implicitly_wait(1)
    actions.send_keys(Keys.END)
    driver.implicitly_wait(1)
    actions.send_keys(Keys.END)
    driver.implicitly_wait(1)
    
    wait = WebDriverWait(driver, 10)

    # p = wait.until(EC.presence_of_element_located((By.CLASS_NAME, 'css-1r5ocs1 ej5yuks0')))

    for i in range(1, 11):
        p = wait.until(EC.presence_of_element_located((By.XPATH, f'/html/body/div/div[1]/div/div/div/div[2]/div/div/section/div[{i}]/div[1]/div[3]/div[1]')))
        

옵션
[TT037] 보들쫀쫀 끈나시_단가라FREE 직진
사이즈
정사이즈예요
퀄리티
아주 만족해요
색감
화면과 비슷해요
정보
159cm39kg상의 44
옵션
핑크(민자)Free 직진
사이즈
정사이즈예요
퀄리티
아주 만족해요
색감
화면과 비슷해요
정보
163cm52kg상의 44
옵션
블랙(민자)Free 직진
사이즈
정사이즈예요
퀄리티
아주 만족해요
색감
화면과 비슷해요
정보
161cm63kg상의 66
옵션
[TT037] 보들쫀쫀 끈나시_단가라FREE 직진
사이즈
정사이즈예요
퀄리티
아주 만족해요
색감
화면과 비슷해요
정보
165cm53kg상의 55
옵션
[TT037] 보들쫀쫀 끈나시_단가라FREE 직진
사이즈
정사이즈예요
퀄리티
보통이에요
색감
화면과 비슷해요
정보
164cm53kg상의 55
옵션
[TT037] 보들쫀쫀 끈나시_단가라FREE 직진
사이즈
정사이즈예요
퀄리티
아주 만족해요
색감
화면과 비슷해요
정보
158cm54kg상의 55
옵션
[TT037] 보들쫀쫀 끈나시_화이트(골지)FREE 직진
사이즈
정사이즈예요
퀄리티
아주 만족해요
색감
화면과 비슷해요
정보
160cm54kg상의 66
옵션
[TT037] 보들쫀쫀 끈나시_단가라FREE 직진
사이즈
정사이즈예요
퀄리티
아주 만족해요
색감
화면과 비슷해요
정보
165cm54kg상의 66
옵션
[TT037] 보들쫀쫀 끈나시_단가라FREE 직진
사이즈
정사이즈예요
퀄리티
아주 만족해요
색감
화면과 비슷해요
옵션
[TT037] 보들쫀쫀 끈나시_그레이(골지)FREE 직진
사이즈
정사이즈예요
퀄리티
아주 만족해요
색감
화면과 비슷해요


In [70]:
with webdriver.Chrome(service=Service(ChromeDriverManager().install())) as driver:
    driver.get("https://zigzag.kr/catalog/products/112538672?tab=review")
    actions = driver.find_element(By.CSS_SELECTOR, 'body')
    actions.send_keys(Keys.END)
    driver.implicitly_wait(1)
    
    wait = WebDriverWait(driver, 10)

    name = wait.until(EC.presence_of_element_located((By.CLASS_NAME, 'BODY_15.REGULAR.css-1qjogoj.e1wgb8lp0')))
    print(name.text)

    price = wait.until(EC.presence_of_element_located((By.CLASS_NAME, 'css-14j45be.eizm2tm0')))
    print(price.text)

    img_url = wait.until(EC.presence_of_element_located((By.CLASS_NAME, 'show-skeleton.css-12ywniv')))
    print(img_url.find_element(By.TAG_NAME, 'picture').find_element(By.TAG_NAME, 'img').get_attribute('src'))

[빅사이즈추가/워터밤][16만장돌파] 옆셔링 캡내장 슬리브리스
41,700
https://cf.product-image.s.zigzag.kr/original/d/2024/6/7/7416_202406071201310448_58975.jpeg?width=400&height=400&quality=80&format=webp


In [52]:
with webdriver.Chrome(service=Service(ChromeDriverManager().install())) as driver:
    driver.get("https://zigzag.kr/catalog/products/112538672?tab=review")
    actions = driver.find_element(By.CSS_SELECTOR, 'body')
    actions.send_keys(Keys.END)
    driver.implicitly_wait(1)
    purchase_button = driver.find_element(By.CLASS_NAME, 'BODY_17.BOLD.css-hbld7k.e1yh52zv0')
    ActionChains(driver).click(purchase_button).perform()
    
    wait = WebDriverWait(driver, 10)
    color_table = wait.until(EC.presence_of_element_located((By.CLASS_NAME, 'css-0.e1u2d7n04')))
    color_table_button = driver.find_element(By.CLASS_NAME, 'BODY_15.SEMIBOLD.css-utqis4.e1cn5bmz0')
    color_list = color_table.find_elements(By.TAG_NAME, 'li')

    for color_tag in color_list:
        time.sleep(0.01)
        color = color_tag.text
        color_tag.click()
        time.sleep(1)

        size_list = wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, 'css-1ef2f7p.ehwmh5c10')))
        print(color)

        for size_tag in size_list:
            size_info = size_tag.text
            time.sleep(1)
            size_info = size_info.split("\n")
            size = size_info[0]
            price = size_info[1]
            print("\t", size, price)
        
        color_table_button.click()
        color_table = wait.until(EC.presence_of_element_located((By.CLASS_NAME, 'css-0.e1u2d7n04')))
        color_list = color_table.find_elements(By.TAG_NAME, 'li')

        time.sleep(5)


아이보리
	 one size 13,900원
	 L 17,900원


StaleElementReferenceException: Message: stale element reference: stale element not found in the current frame
  (Session info: chrome=126.0.6478.127); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#stale-element-reference-exception
Stacktrace:
0   chromedriver                        0x0000000100d92a80 chromedriver + 4385408
1   chromedriver                        0x0000000100d8b38c chromedriver + 4354956
2   chromedriver                        0x00000001009a8b0c chromedriver + 281356
3   chromedriver                        0x00000001009acfd8 chromedriver + 298968
4   chromedriver                        0x00000001009aee58 chromedriver + 306776
5   chromedriver                        0x00000001009aeed0 chromedriver + 306896
6   chromedriver                        0x00000001009e62a0 chromedriver + 533152
7   chromedriver                        0x00000001009e1734 chromedriver + 513844
8   chromedriver                 